In [56]:
infile = './data/GLA-1R46.chimerax'
outfile = './data/GLA-1R46.cxc'

In [53]:
import xml.etree.ElementTree as ET

In [59]:
def convert_chimerax2cxc(infile, outfile):
    myTree = ET.parse(infile)
    ChimeraPuppet = myTree.getroot()
    pdb = ChimeraPuppet[0][0].attrib['id']
    lines = [f'open {pdb}']
    commands = ChimeraPuppet[1]
    for command in commands:
        new_cmd = convert_command(command.text)
        if new_cmd:
            lines.append(new_cmd)
    with open(outfile, 'w') as f:
        f.write('\n'.join(lines))

In [60]:
convert_chimerax2cxc(infile, outfile)

In [20]:
mytree = ET.parse(infile)
ChimeraPuppet = mytree.getroot()

In [27]:
pdb = ChimeraPuppet[0][0].attrib['id']

In [52]:
commands = ChimeraPuppet[1]
for command in commands:
    new_cmd = convert_command(command.text)
    print(new_cmd)

label /A:237@NZ atoms text "Lys-237"
label /A:139@CA atoms text "Asn-139"
label /A:186@CE1 atoms text "His-186"
label /A:391@NZ atoms text "Lys-391"
label /A:393@NZ atoms text "Lys-393"
label /A:127@NZ atoms text "Lys-127"
label /A:314@NZ atoms text "Lys-314"
label /A:371@OG atoms text "Ser-371"
label /A:326@NZ atoms text "Lys-326"
label /A:402@CZ atoms text "Arg-402"
label /A:240@NZ atoms text "Lys-240"
label /A:308@NZ atoms text "Lys-308"
label /A:192@CA atoms text "Asn-192"
label /B:237@NZ atoms text "Lys-237"
label /B:139@CA atoms text "Asn-139"
label /B:186@CE1 atoms text "His-186"
label /B:391@NZ atoms text "Lys-391"
label /B:393@NZ atoms text "Lys-393"
label /B:127@NZ atoms text "Lys-127"
label /B:314@NZ atoms text "Lys-314"
label /B:371@OG atoms text "Ser-371"
label /B:326@NZ atoms text "Lys-326"
label /B:402@CZ atoms text "Arg-402"
label /B:240@NZ atoms text "Lys-240"
label /B:308@NZ atoms text "Lys-308"
label /B:192@CA atoms text "Asn-192"
None
None
None
color /A:237@CE,NZ cy

In [61]:
def convert_command(command):
    elements = command.split(' ')
    cmd = ''
    if elements[0] == 'setattr':
        level = elements[1]
        if elements[2] == 'label':
            cmd = 'label'
    elif elements[0] == 'color':
        cmd = 'color'
    elif elements[0] == 'surface':
        cmd = 'surface'
    elif elements[0] == 'surftrans':
        cmd = 'transparency'
    
    if cmd == 'label':
        value = elements[3]
        specifier = elements[4]
        specifier = _convert_specifier(specifier)
        level = _convert_level(level)
        return f'label {specifier} {level} text {value}'
    elif cmd == 'color':
        colorname = elements[1]
        specifier = elements[2]
        specifier = _convert_specifier(specifier)
        return f'color {specifier} {colorname}'
    elif cmd == 'surface':
        return 'surface'
    elif cmd == 'transparency':
        percent = elements[1]
        return f'transparency {percent}'


def _convert_specifier(specifier):
    # Assumes a format like ":88.A@OG" 
    index = specifier.find('.')
    if index < 0:
        return specifier
    residues = specifier[:index]
    chain = specifier[index+1]
    atoms = specifier[index+2:]
    return f'/{chain}{residues}{atoms}'


def _convert_level(level):
    # a | r | m | b | p | g
    if level == 'a':
        return 'atoms'
    elif level == 'r':
        return 'residues'
    elif level == 'b':
        return 'bonds'
    elif level == 'p':
        return 'pseudobonds'
    else:  # m or g
        print(f'Unable to process "{level}"')
        return level
